In [1]:
import ast
import pandas as pd
from pymongo import MongoClient
from selenium import webdriver
# from selenium.webdriver.common.keys import Keys 
from time import sleep
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

**2). Написать программу, которая собирает «Хиты продаж» с сайта техники mvideo и складывает данные в БД. Магазины можно выбрать свои. Главный критерий выбора: динамически загружаемые товары**

In [2]:
client = MongoClient('mongodb://127.0.0.1:27017')

db_name = 'litres'

In [3]:
db_list = client.list_database_names()
if db_name in db_list:
    client.drop_database(db_name)

In [4]:
db = client[db_name]
docs = db.docs

In [5]:
options = webdriver.ChromeOptions()
options.binary_location = r'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe'
chrome_driver_binary = r"C:/Users/TOSHIBA/Downloads/chromedriver_win32/chromedriver.exe"

In [6]:
driver = webdriver.Chrome(executable_path=chrome_driver_binary, options=options)
driver.get("https://www.litres.ru")

In [7]:
SCROLL_PAUSE_TIME = 2.5
pages = 0
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    
    sleep(SCROLL_PAUSE_TIME)
    button = driver.find_element_by_id('arts_loader_button')
    # button = WebDriverWait(driver, 5).until(
    #         EC.presence_of_element_located((By.ID, 'arts_loader_button'))
    #    )
    if button.text != '':
        button.click()  
        pages += 1
    if pages == 5:
        break
        
# https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python

In [8]:
def add_to_db(db, item_info):
    is_link_in = db.find_one({'art_id': item_info['art_id']})
    if is_link_in == None:
        result = db.insert_one(item_info)

In [9]:
columns = ['art_id:', 'author_id:', 'alt:',
           'author:', 'type:', 'mem:', 'drm:',
           'price:', 'available:', 'available_date:',
           'can_preorder:', 'main_book_id:',
           'promo_price:', 'purchased:']

In [10]:
df = pd.DataFrame({})
for cover in driver.find_elements_by_class_name('cover'):
    data_obj = cover.get_attribute('data-obj')
    try:
        for column in columns:
            data_obj = data_obj.replace(column, " '{}':".format(column[:-1]))
        data_obj = data_obj.replace("promo_ 'price'", "'promo_price'")
        item_info = ast.literal_eval(data_obj)
        df = df.append(item_info, ignore_index=True)
        add_to_db(docs, item_info)
    except:
        continue

In [11]:
subset = df.columns[df.columns != 'main_book_id']
df.drop_duplicates(subset=subset, inplace=True)

In [12]:
df.head()

,alt,art_id,author,author_id,available,available_date,can_preorder,drm,main_book_id,mem,price,promo_price,type,purchased
0,Другая правда. Том 2,43427366,Александра Маринина,54149,6,10 сентября,1,0,43427366,text,299.00,209.30,0,NaN
1,Казнь без злого умысла,9368246,Александра Маринина,54149,1,NaN,,0,43427366,text,229.00,229.00,0,NaN
2,Горький квест. Том 1,36306148,Александра Маринина,54149,1,NaN,,0,43427366,text,299.00,299.00,0,NaN
3,Тревожная весна 45-го,42935946,Валерий Шарапов,18709880,1,NaN,,0,43427366,text,219.00,219.00,0,NaN
4,Папа с прицепом,43419352,Николай Леонов,45453,1,NaN,,0,43427366,text,299.00,299.00,0,NaN


In [13]:
df.shape

(167, 14)

In [14]:
docs.estimated_document_count()

167